# Benchmark for vibration of a string

Assumptions made:
- string is treated as perfectly flexible (no stiffness)
- fixed-fixed condition so that reflections are perfectly inverted

Goal: simulate the kink resulting from plucking the string

Constituitive equation: $\frac{\partial^2 y}{\partial t^2} = c^2 \frac{\partial^2 y}{\partial x^2}$

where,
$
y = displacement\:of\:the\:string\:from\:equilibirum \\
x = position\:along\:the\:string \\
t = time \\
c = \sqrt{\frac{T}{\mu}} = wave\:speed \\
T = tension \\
\mu = mass\:per\:unit\:length \\
$

Citation(s):
$@article\:{giordano1998physics,
  title={The physics of vibrating strings},
  author={Giordano, Nicholas and Gould, Harvey and Tobochnik, Jan}, 
  journal={Computers in Physics}, 
  volume={12},
  number={2}, 
  pages={138--145}, 
  year={1998},
  publisher={American Institute of Physics}
}$

In [1]:
using CombinatorialSpaces
using CombinatorialSpaces.DiscreteExteriorCalculus: inv_hodge_star
using DifferentialEquations
using CairoMakie

# dual subdivision 
function dual(s::EmbeddedDeltaSet2D{O, P}) where {O, P}
  sd = EmbeddedDeltaDualComplex2D{O, eltype(P), P}(s)
  subdivide_duals!(sd, Circumcenter())
  sd
end

┌ Info: Precompiling CairoMakie [13f3f980-e62b-5c42-98c6-ff1f3baf88f0]
└ @ Base loading.jl:1317


dual (generic function with 1 method)

## Define and plot mesh

In [3]:
# Define mesh
s = EmbeddedDeltaSet2D("meshes/string.obj")
sd = dual(s)

# figure, axis, ob = wireframe(s)
# # axis.aspect = AxisAspect(10/1)
# figure

V,vertex_center,point
1,1,"Float32[0.0, 0.0, 0.0]"
2,2,"Float32[0.1, 0.0, 0.0]"
3,3,"Float32[0.15, 0.0, 0.0]"
4,4,"Float32[0.2, 0.0, 0.0]"
5,5,"Float32[0.25, 0.0, 0.0]"
6,6,"Float32[0.3, 0.0, 0.0]"
7,7,"Float32[0.35, 0.0, 0.0]"
8,8,"Float32[0.4, 0.0, 0.0]"
9,9,"Float32[0.45, 0.0, 0.0]"
10,10,"Float32[0.5, 0.0, 0.0]"
